In [13]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [14]:

def get_gpu_info():
    if torch.cuda.is_available():
        gpu_info = {}
        props = torch.cuda.get_device_properties(0)
        gpu_info['GPU Name'] = props.name
        gpu_info['Total Memory (MB)'] = props.total_memory // (1024 * 1024)
        gpu_info['CUDA Capability'] = f"{props.major}.{props.minor}"
        gpu_info['Multiprocessor Count'] = props.multi_processor_count

        return gpu_info
    else:
        return "No GPU available"

if __name__ == "__main__":
    gpu_info = get_gpu_info()
    if isinstance(gpu_info, dict):
        for key, value in gpu_info.items():
            print(f"{key}: {value}")
    else:
        print(gpu_info)


GPU Name: NVIDIA GeForce RTX 3090
Total Memory (MB): 24252
CUDA Capability: 8.6
Multiprocessor Count: 82


In [15]:
# Import the required libraries
# For this example we will use pytorch to manage the construction of the neural networks and the training
# torchvision is a module that is part of pytorch that supports vision datasets and it will be where we will source the mnist - handwritten digits - data

from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils



In [2]:
# Setting a seed will determine which data elements are selected. To replicate results keep the same seed.
manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)


Random Seed:  9772


In [3]:
# This is a check if there is a gpu available for training. At the moment we are assuming that it is not available.
torch.cuda.is_available()


True

In [4]:

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Parameters
cudnn.benchmark = True
ngpu = 1 if torch.cuda.is_available() else 0
nz = 100  # Latent space matrix width
ngf = 64  # Generator matrix shape
ndf = 64  # Discriminator matrix shape
nc = 1  # Number of color channels
batch_size = 64  # Number of samples per pass
workers = 4  # Number of CPU workers for the dataset


Using device: cuda


In [5]:
dataset = dset.MNIST(root='data', download=True,
                      transform=transforms.Compose([
                          transforms.Resize(64),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5,), (0.5,)),
                      ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=int(workers))


Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [6]:


# custom weights initialization called on netG and netD
# The weights will need to be initialised based on the layer type to some value before training. These could be imported from past training steps.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)


In [7]:
# This is the bulk of the neural network definition for the Generator.
# The init sets up the layers and connecting activation functions.
# The forward function processes the data through the layers
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(
                self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output


netG = Generator(ngpu).to(device)
netG.apply(weights_init)
print(netG)


Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [8]:
# This is the bulk of the neural network definition for the Discrimator.
# The init sets up the layers and connecting activation functions.
# The forward function processes the data through the layers
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(
                self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)
    
netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
print(netD)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [9]:
# Set the loss function from pytorches established modules
criterion = nn.BCELoss()

# Set up the initial noise of the latent space to sample from.
# Set the label of a real and fake sample to 0,1
fixed_noise = torch.randn(64, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

# Create the optimiser which will dynamically change the parameters of the learning function over time to imporve the training process
optimizerD = optim.Adam(netD.parameters(), lr=0.0005, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0005, betas=(0.5, 0.999))


In [10]:
for epoch in range(1):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        
        # Set the descrimator to forget any gradients.
        netD.zero_grad()
        # Get a sample of real handwritten digits and label them as 1 - all real
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, dtype=real_cpu.dtype, device=device)
        # Pass the sample through the descrimator
        output = netD(real_cpu)
        # measure the error
        errD_real = criterion(output, label)
        # Calculate the gradients of each layer of the network
        errD_real.backward()
        # Get the average of the output across the batch
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        # pass the noise through the generator layers
        fake = netG(noise)
        # set the labels to all 0 - fake
        label.fill_(fake_label)
        # ask the descrimator to judge the fake images
        output = netD(fake.detach())
        # measure the error
        errD_fake = criterion(output, label)
        # Calculate the gradients 
        errD_fake.backward()
        # Get the average output across the batch again
        D_G_z1 = output.mean().item()
        # Get the error
        errD = errD_real + errD_fake
        # Run the optimizer to update the weights
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        # Set the gradients of the generator to zero
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # get the judgements from the descrimator of the generator output is fake
        output = netD(fake)
        # calculate the error
        errG = criterion(output, label)
        # update the gradients
        errG.backward()
        # Get the average of the output across the batch
        D_G_z2 = output.mean().item()
        # update the weights
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, 1, i, len(dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        # every 100 steps save a real sample and a fake sample for comparison
        if i % 100 == 0:
            vutils.save_image(real_cpu, 'real_samples.png', normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(), 'fake_samples_epoch_%03d.png' % epoch, normalize=True)


[0/1][0/938] Loss_D: 1.7782 Loss_G: 12.3799 D(x): 0.5280 D(G(z)): 0.5843 / 0.0000
[0/1][1/938] Loss_D: 4.8980 Loss_G: 14.1061 D(x): 0.9999 D(G(z)): 0.9859 / 0.0000
[0/1][2/938] Loss_D: 0.4470 Loss_G: 15.9369 D(x): 0.9871 D(G(z)): 0.3105 / 0.0000
[0/1][3/938] Loss_D: 0.7976 Loss_G: 5.3191 D(x): 0.7042 D(G(z)): 0.0244 / 0.0083
[0/1][4/938] Loss_D: 5.1032 Loss_G: 22.8838 D(x): 0.9746 D(G(z)): 0.9865 / 0.0000
[0/1][5/938] Loss_D: 0.2861 Loss_G: 22.6269 D(x): 0.8507 D(G(z)): 0.0000 / 0.0000
[0/1][6/938] Loss_D: 0.3236 Loss_G: 7.4387 D(x): 0.8885 D(G(z)): 0.0022 / 0.0017
[0/1][7/938] Loss_D: 6.9209 Loss_G: 27.0062 D(x): 0.9819 D(G(z)): 0.9975 / 0.0000
[0/1][8/938] Loss_D: 1.4033 Loss_G: 26.0488 D(x): 0.4960 D(G(z)): 0.0000 / 0.0000
[0/1][9/938] Loss_D: 0.2062 Loss_G: 10.7406 D(x): 0.9460 D(G(z)): 0.0001 / 0.0001
[0/1][10/938] Loss_D: 4.2189 Loss_G: 30.2608 D(x): 0.9975 D(G(z)): 0.9705 / 0.0000
[0/1][11/938] Loss_D: 0.6301 Loss_G: 35.2016 D(x): 0.7654 D(G(z)): 0.0000 / 0.0000
[0/1][12/938] Lo